In [1]:
"""We import the py file which contains all the functions we created to preprocess the data before classifying them
(especially to link the article to its category and studied language)"""
from benderrule_tools import *
import json

In [ ]:
liste_chemins_annotes_lrec = chemin_corpus("lrec_annotes2/*/*","en","lrec")
liste_lang = get_list_lang("en")

dico_br_app_lrec_a = br_appliquee(liste_chemins_annotes_lrec,liste_lang,"english")
data_annote_all = dico_br_appliquee_phrases(liste_chemins_annotes_lrec,liste_lang, dico_br_app_lrec_a)

with open("data_classif1_en420f.json") as f:
    data_annote = json.load(f)
textes_en = [x[0] for x in data_annote]
classes_en = [x[1] for x in data_annote]

In [ ]:
classifiers_report(textes_en)

In [ ]:
data_annote = [["ACL", "420-acl_annotation_bender.json"], ["LREC", "550-lrec_annotation_bender.json"]]

with open("corpus_lremap.json")as f:
    names_LRE = json.load(f)
    
resultats_conf(data_annote, names_LRE)

In [6]:
from sklearn.linear_model import LogisticRegression
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

classifiers = []
for pen in ["l1", "l2"]:
    classifiers.append(["LogReg_saga_pen=%s"%pen, 
                        LogisticRegression(multi_class="auto", solver="saga", 
                                           max_iter=500, class_weight="balanced",
                                          penalty = pen)])

    
vectorizers=[
    ["Tfidf",TfidfVectorizer()],
    ["Count", CountVectorizer()]
]
for nom_classif, algo in classifiers:
  print(nom_classif)
  for name, V in vectorizers:
    X1_en = V.fit_transform(textes_en)
    Train_V = V.fit(textes_en)
    Y1_en = classes_en
    X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(X1_en, Y1_en, test_size=0.3, random_state=0)
    start = time.perf_counter()
    clf_en = algo.fit(X_train_en, y_train_en)
    end = time.perf_counter()
    print(name, "... %.2f seconds"%(round(end-start,3)))
    pred_en = clf_en.predict(X_test_en)

    conf_matrix_en = confusion_matrix(y_test_en, pred_en)
    report_en = classification_report(y_test_en,pred_en)
    print(report_en)


LogReg_saga_pen=l1


/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Tfidf ... 1.53 seconds
               precision    recall  f1-score   support

    appliquée       0.92      0.96      0.94       704
non-appliquée       0.47      0.32      0.38        84

     accuracy                           0.89       788
    macro avg       0.70      0.64      0.66       788
 weighted avg       0.87      0.89      0.88       788



/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Count ... 2.68 seconds
               precision    recall  f1-score   support

    appliquée       0.94      0.92      0.93       704
non-appliquée       0.42      0.49      0.45        84

     accuracy                           0.87       788
    macro avg       0.68      0.70      0.69       788
 weighted avg       0.88      0.87      0.88       788

LogReg_saga_pen=l2


/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Tfidf ... 0.41 seconds
               precision    recall  f1-score   support

    appliquée       0.94      0.02      0.04       704
non-appliquée       0.11      0.99      0.19        84

     accuracy                           0.13       788
    macro avg       0.52      0.51      0.12       788
 weighted avg       0.85      0.13      0.06       788

Count ... 0.42 seconds
               precision    recall  f1-score   support

    appliquée       0.93      0.95      0.94       704
non-appliquée       0.52      0.44      0.48        84

     accuracy                           0.90       788
    macro avg       0.73      0.70      0.71       788
 weighted avg       0.89      0.90      0.89       788



/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [7]:
with open("corpus_lremap.json")as f:
    names_LRE = json.load(f)
    
stats_LRE = {}
data_annote = [["ACL", "420-acl_annotation_bender.json"], ["LREC", "550-lrec_annotation_bender.json"]]
resultats = {}
for conf_name, json_path in data_annote:
    print(conf_name)
    resultats[conf_name] = {"Sentence":[], "Baseline":[], "Baseline_LRE":[], "Sentence_LRE":[]}
    with open(json_path) as f:
        dic_conf = json.load(f)
    for path_article, infos_article in dic_conf.items():
        classe = infos_article["class"]
        if classe =="Non-déductible" or classe =="Non-Aplicable":
            classe="N/A"
        chaine = read_file(path_article)
        has_LRE = False
        for name in names_LRE:
            if name in chaine:
                has_LRE = True
                stats_LRE.setdefault(name,0)
                stats_LRE[name]+=1
        chaine = chaine.lower()
        phrases = chaine.split(".")
        a_tester = []
        for phrase in phrases:
            for mot in phrase.split():
                for l in liste_lang:
                    if mot == l:
                        a_tester.append(phrase)
        if len(a_tester)>0:
            pred_baseline = "Appliquée"
            pred_LRE = pred_baseline
            if classe =="Déductible":#pour avoir aussi les déductibles
                pred_baseline = classe
            X_test2 = Train_V.transform(a_tester)
            Y_pred2 = clf_en.predict(X_test2)
        else:
            pred_baseline = "N/A"
            Y_pred2= [["N/A"]]
            if has_LRE==True:
                pred_LRE = "Déductible"
            else:
                pred_LRE = pred_baseline

        NB_pos = len([x for x in Y_pred2 if x== "appliquée"])
        if "appliquée" in Y_pred2:
            pred_sentence = "Appliquée"
            if classe =="Déductible":#pour avoir aussi les déductibles
                pred_sentence = classe
        else:
            pred_sentence = "N/A"
        pred_sentence_LRE = pred_sentence
        if pred_sentence!="Appliquée":
            if pred_LRE=="Déductible":
                pred_sentence_LRE = pred_LRE
            "Sentence_LRE"
        resultats[conf_name]["Sentence"].append([classe, pred_sentence])
        #, {"Phr_lg":len(a_tester), "Phr_lg_pos":NB_pos}])
        resultats[conf_name]["Baseline"].append([classe, pred_baseline])
        resultats[conf_name]["Baseline_LRE"].append([classe, pred_LRE])
        resultats[conf_name]["Sentence_LRE"].append([classe, pred_sentence_LRE])


ACL
LREC


In [8]:
from scipy.stats import spearmanr

for class_name, dic_res_lrec in resultats["LREC"].items():
    print("-"*20)
    print(class_name+"\n")
    y_test, y_pred = [x[0] for x in dic_res_lrec], [x[1] for x in dic_res_lrec]
    report_lrec = classification_report(y_test, y_pred, output_dict=True)
    print("LREC", str(spearmanr(y_test,y_pred)))

    dic_res_acl = resultats["ACL"][class_name]
    y_test, y_pred = [x[0] for x in dic_res_acl], [x[1] for x in dic_res_acl]
    report_acl = classification_report(y_test, y_pred, output_dict=True)
    print("ACL", str(spearmanr(y_test,y_pred)))

    dic_classes = {"Appliquée":"Applied\t", "Déductible":"Deducible", "N/A":"N/A\t", "macro avg":"macro avg", "weighted avg":"micro avg"}
    for classe in ["Appliquée", "Déductible", "N/A", "macro avg", "weighted avg"]:
        liste_ligne = [dic_classes[classe]+"\t"]
        for this_res in [report_lrec, report_acl]:
            for mesure in ["precision", "recall", "f1-score"]:
                liste_ligne.append(round(this_res[classe][mesure], 3))
            liste_ligne.append(this_res[classe]["support"])
        print(" & ".join([str(x) for x in liste_ligne])+"\\\\")
        


--------------------
Sentence

LREC SpearmanrResult(correlation=0.66363633080393, pvalue=4.218059408516568e-71)
ACL SpearmanrResult(correlation=0.7372822669269066, pvalue=5.007545573363368e-73)
Applied		 & 0.908 & 0.986 & 0.946 & 440 & 0.855 & 0.974 & 0.911 & 230\\
Deducible	 & 1.0 & 0.6 & 0.75 & 20 & 1.0 & 0.286 & 0.444 & 42\\
N/A		 & 0.367 & 0.579 & 0.449 & 38 & 0.448 & 0.707 & 0.549 & 92\\
macro avg	 & 0.569 & 0.541 & 0.536 & 550 & 0.576 & 0.492 & 0.476 & 419\\
micro avg	 & 0.788 & 0.851 & 0.815 & 550 & 0.668 & 0.718 & 0.665 & 419\\
--------------------
Baseline

LREC SpearmanrResult(correlation=0.625036153908324, pvalue=6.092385324772995e-61)
ACL SpearmanrResult(correlation=0.507784425004818, pvalue=7.605421904649439e-29)
Applied		 & 0.894 & 0.993 & 0.941 & 440 & 0.728 & 0.978 & 0.835 & 230\\
Deducible	 & 1.0 & 0.65 & 0.788 & 20 & 1.0 & 0.595 & 0.746 & 42\\
N/A		 & 0.396 & 0.5 & 0.442 & 38 & 0.435 & 0.402 & 0.418 & 92\\
macro avg	 & 0.572 & 0.536 & 0.543 & 550 & 0.541 & 0.494 & 0.5

/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rundimeco/anacon

In [9]:
print(stats_LRE)

{'ESO': 21, 'UE': 92, 'RTE': 23, 'DUC': 70, 'Semi': 105, 'EAGLE': 20, 'NSP': 10, 'CID': 40, 'SUC': 10, 'AMI': 56, 'TUT': 8, 'AAC': 191, 'CSI': 25, 'Turkish': 48, 'GMA': 6, 'DMC': 3, 'DCR': 4, 'FEAT': 4, 'ACD': 6, 'DiSCo': 1, 'COW': 8, 'RSC': 4, 'BAT': 2, 'WSJ': 38, 'Wall Street Journal Corpus': 1, 'with parsing': 2, 'Unified Medical Language System': 8, 'Le Monde': 6, 'C 3': 24, 'Topic Detection and Tracking': 4, 'Switchboard': 17, 'NYT': 7, 'ELC': 3, 'PropBank': 20, 'British National Corpus': 140, 'BNC': 165, 'Argo': 1, 'UMLS': 24, 'ATB': 5, 'LDC2006T13': 2, 'MALLET': 3, 'PDT': 17, 'SUMO': 16, 'PMT': 3, 'SDG': 2, 'Automatic Content Extraction': 8, 'NUCLE': 4, 'RNC': 4, 'ukWaC': 5, 'TAC KBP': 6, 'MPQA': 14, 'MIM': 5, 'ODIN': 1, 'Audio visual': 1, 'Unofficial': 2, 'CSLU': 1, 'CHILDES': 3, 'CRPC': 2, 'PHOC': 1, 'MASC': 4, 'Le Robert': 1, 'Trésor de la Langue Française informatisé': 1, 'TLFi': 1, 'Suggested Upper Merged Ontology': 8, 'SAMA': 1, 'ESTER': 3, 'ERJ': 1, 'Russian National Corp

In [10]:
#### Tester vectoriseurs
algo = LogisticRegression(multi_class="auto", solver="lbfgs", max_iter=500, class_weight="balanced")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
#Balanced est indispensable
classifiers = [
    #["base+bal", LogisticRegression(multi_class="auto", solver="lbfgs", class_weight="balanced")
    #],
    #["base+bal+iter", LogisticRegression(multi_class="auto", solver="lbfgs", max_iter=1000, class_weight="balanced")
    #],
        ["base+bal+saga", LogisticRegression(multi_class="auto", solver="saga", class_weight="balanced")
    ],
    ["base+bal+iter+saga", LogisticRegression(multi_class="auto", solver="saga", max_iter=1000, class_weight="balanced")
    ],
    #["LogisticRegr_Bal_C=0.01", LogisticRegression(multi_class="auto", solver="saga", C=0.01, n_jobs=2, max_iter=1000, class_weight="balanced")],
    ["LogisticRegr_Bal_C=0.05", LogisticRegression(multi_class="auto", solver="saga", C=0.05, n_jobs=2, max_iter=1000, class_weight="balanced")],
    ["Decision Tree", DecisionTreeClassifier()],
    #["MNB", MultinomialNB()],
    #["RF_depth=10_estimators=100", RandomForestClassifier(max_depth=10, n_estimators = 100, random_state=0)],
]
    

#V = CountVectorizer(ngram_range=(3,5), analyzer="char")
list_vectorizer = [
    ["Count 1-1\t", CountVectorizer()],
    ["Count 3-7--char", CountVectorizer(ngram_range=(3,7), analyzer="char")],
    #["tfidf", TfidfVectorizer()],
    #["tfidf 3-7--char", TfidfVectorizer(ngram_range=(3,7), analyzer="char")],
]
list_res = []
for name, clf in classifiers:
  for nom, this_V in list_vectorizer:
    X1_en = this_V.fit_transform(textes_en)
    Train_V = this_V.fit(textes_en)
    Y1_en = classes_en
    
    X_train_en, X_test_en, y_train_en, y_test_en = train_test_split(X1_en, Y1_en, test_size=0.3, random_state=0)
    start = time.perf_counter()
    clf_new = clf.fit(X_train_en, y_train_en)
    end = time.perf_counter()
    #print("... %.2f seconds"%(round(end-start,2)))
    pred_en = clf_new.predict(X_test_en)

    conf_matrix_en = confusion_matrix(y_test_en, pred_en)
    #print("Classifieur 1\n",conf_matrix_en) #na/app 229/3 2/9
    report_en = classification_report(y_test_en,pred_en,output_dict=True)
    print(name, nom, report_en["macro avg"]["f1-score"])

    list_res.append([report_en["macro avg"]["f1-score"], name, nom] )
for a in sorted(list_res):
    print(a)

/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


base+bal+saga Count 1-1	 0.7080934840065763


/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


base+bal+saga Count 3-7--char 0.6923557429530726
base+bal+iter+saga Count 1-1	 0.7049336036677809


/home/rundimeco/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


base+bal+iter+saga Count 3-7--char 0.6819094541163807
LogisticRegr_Bal_C=0.05 Count 1-1	 0.649761449640232
LogisticRegr_Bal_C=0.05 Count 3-7--char 0.6883395162784177
Decision Tree Count 1-1	 0.6188954641539373
Decision Tree Count 3-7--char 0.6295094957605853
[0.6188954641539373, 'Decision Tree', 'Count 1-1\t']
[0.6295094957605853, 'Decision Tree', 'Count 3-7--char']
[0.649761449640232, 'LogisticRegr_Bal_C=0.05', 'Count 1-1\t']
[0.6819094541163807, 'base+bal+iter+saga', 'Count 3-7--char']
[0.6883395162784177, 'LogisticRegr_Bal_C=0.05', 'Count 3-7--char']
[0.6923557429530726, 'base+bal+saga', 'Count 3-7--char']
[0.7049336036677809, 'base+bal+iter+saga', 'Count 1-1\t']
[0.7080934840065763, 'base+bal+saga', 'Count 1-1\t']
